# Sucess Code (100value)

# Final Version (Detect Null without storing it, only store valid data)

In [11]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import regexp_replace, col
import json
import base64

class KafkaStreamConsumer:
    def __init__(self, topic, bootstrap_servers, group_id, hdfs_path, max_records):
        self.topic = topic
        self.bootstrap_servers = bootstrap_servers
        self.group_id = group_id
        self.hdfs_path = hdfs_path
        self.max_records = max_records
        self.record_count = 0

        self.consumer = KafkaConsumer(
            self.topic,
            bootstrap_servers=self.bootstrap_servers,
            group_id=self.group_id,
            value_deserializer=lambda x: json.loads(x.decode('utf-8'))
        )

        # Initialize Spark session
        self.spark = SparkSession.builder \
            .appName("KafkaToSpark") \
            .getOrCreate()
        
        # Define schema as StructType
        self.schema = StructType([
            StructField("Price", StringType(), True),
            StructField("Shape", StringType(), True),
            StructField("Carat", StringType(), True),
            StructField("Clarity", StringType(), True),
            StructField("Color", StringType(), True),
            StructField("Cut", StringType(), True),
            StructField("Polish", StringType(), True),
            StructField("Symmetry", StringType(), True),
            StructField("Fluorescence", StringType(), True),
            StructField("Certificate No", StringType(), True),
            StructField("Lab", StringType(), True),
            StructField("View", StringType(), True)
        ])

    def preprocess_data(self, data):
        print(f"Preprocessing data: {data}")  # Debug print
        
        # Base64 decode the data
        base64_str = data.get('$binary', {}).get('base64', '')
        if base64_str:
            decoded_bytes = base64.b64decode(base64_str)
            decoded_str = decoded_bytes.decode('utf-8')
            data = json.loads(decoded_str)
        
        # Convert the dictionary to a Spark DataFrame
        df = self.spark.createDataFrame([data], schema=self.schema)
        
        # Perform preprocessing
        df = df.withColumn("Price", regexp_replace(col("Price"), "RM|,", "")) \
               .withColumn('Price', col('Price').cast(DoubleType())) \
               .withColumn('Carat', col('Carat').cast(DoubleType())) \
               .drop('Cut', 'Lab', 'View', 'Certificate No')

        # Print DataFrame schema and data
        df.printSchema()
        df.show(truncate=False)
        
        return df

    def start_consuming(self):
        print("Starting to consume messages from Kafka...")
        try:
            for message in self.consumer:
                if self.record_count >= self.max_records:
                    print(f"Reached {self.max_records} records. Stopping consumption.")
                    break
                preprocessed_df = self.preprocess_data(message.value)
                if not preprocessed_df.rdd.isEmpty():  # Check if DataFrame is empty
                    self.process_message(preprocessed_df)
                else:
                    print("Empty DataFrame, skipping...")
                self.record_count += 1
                print(f"Processed record number: {self.record_count}")
        except KeyboardInterrupt:
            print("Consumption stopped by user.")
        except Exception as e:
            print(f"Error during consumption: {e}")
        finally:
            self.cleanup()

    def process_message(self, df):
        # Save DataFrame to HDFS
        self.save_to_file(df)

    def save_to_file(self, df):
        try:
            # Save DataFrame to JSON file in HDFS
            df.write.mode('append').json(self.hdfs_path)
            print("Data saved to file")
        except Exception as e:
            print(f"Error saving data to HDFS: {e}")

    def cleanup(self):
        self.consumer.close()
        self.spark.stop()
        print("Consumer and Spark session closed.")

if __name__ == "__main__":
    kafka_topic = 'diamond'
    kafka_bootstrap_servers = ['localhost:9092']
    kafka_group_id = 'my-group'
    hdfs_path = 'hdfs://localhost:9000/user/student/de-dir'  # Specify your HDFS path here
    max_records = 1320  # Number of records to process

    consumer = KafkaStreamConsumer(
        topic=kafka_topic,
        bootstrap_servers=kafka_bootstrap_servers,
        group_id=kafka_group_id,
        hdfs_path=hdfs_path,
        max_records=max_records
    )

    consumer.start_consuming()


Starting to consume messages from Kafka...
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDIxMmQxZGMwOWY1N2Q4NTEzIn0sICJQcmljZSI6ICJSTSAyLDQ2Ny4wNSIsICJTaGFwZSI6ICJQZWFyIiwgIkNhcmF0IjogIjAuNTAwIiwgIkNsYXJpdHkiOiAiVlZTMiIsICJDb2xvciI6ICJKIiwgIkN1dCI6ICJWRyIsICJQb2xpc2giOiAiRVgiLCAiU3ltbWV0cnkiOiAiVkciLCAiRmx1b3Jlc2NlbmNlIjogIk5vbmUiLCAiQ2VydGlmaWNhdGUgTm8iOiAiKioqKioqNjEzMiIsICJMYWIiOiAiR0lBIiwgIlZpZXciOiAiTW9yZSBJbmZvcm1hdGlvbiJ9', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|2467.

Data saved to file
Processed record number: 13
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDIxMmQxZGMwOWY1N2Q4NTIwIn0sICJQcmljZSI6ICJSTSAyLDUxMi41NiIsICJTaGFwZSI6ICJQZWFyIiwgIkNhcmF0IjogIjAuMzAwIiwgIkNsYXJpdHkiOiAiSUYiLCAiQ29sb3IiOiAiRyIsICJDdXQiOiAiRVgiLCAiUG9saXNoIjogIkVYIiwgIlN5bW1ldHJ5IjogIkVYIiwgIkZsdW9yZXNjZW5jZSI6ICJOb25lIiwgIkNlcnRpZmljYXRlIE5vIjogIioqKioqKjI5MzEiLCAiTGFiIjogIkdJQSIsICJWaWV3IjogIk1vcmUgSW5mb3JtYXRpb24ifQ==', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|2

Data saved to file
Processed record number: 14
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDIxMmQxZGMwOWY1N2Q4NTIxIn0sICJQcmljZSI6ICJSTSAyLDUxMi41NiIsICJTaGFwZSI6ICJQZWFyIiwgIkNhcmF0IjogIjAuMzEwIiwgIkNsYXJpdHkiOiAiVlZTMSIsICJDb2xvciI6ICJHIiwgIkN1dCI6ICJFWCIsICJQb2xpc2giOiAiRVgiLCAiU3ltbWV0cnkiOiAiRVgiLCAiRmx1b3Jlc2NlbmNlIjogIk5vbmUiLCAiQ2VydGlmaWNhdGUgTm8iOiAiKioqKioqMTcxMCIsICJMYWIiOiAiR0lBIiwgIlZpZXciOiAiTW9yZSBJbmZvcm1hdGlvbiJ9', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|2

Data saved to file
Processed record number: 19
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDIxMmQxZGMwOWY1N2Q4NTI2In0sICJQcmljZSI6ICJSTSAyLDUyNS43OCIsICJTaGFwZSI6ICJQZWFyIiwgIkNhcmF0IjogIjAuMzAwIiwgIkNsYXJpdHkiOiAiVlZTMSIsICJDb2xvciI6ICJGIiwgIkN1dCI6ICJFWCIsICJQb2xpc2giOiAiRVgiLCAiU3ltbWV0cnkiOiAiVkciLCAiRmx1b3Jlc2NlbmNlIjogIk5vbmUiLCAiQ2VydGlmaWNhdGUgTm8iOiAiKioqKioqMjU0NSIsICJMYWIiOiAiR0lBIiwgIlZpZXciOiAiTW9yZSBJbmZvcm1hdGlvbiJ9', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|2

Data saved to file
Processed record number: 21
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDIxMmQxZGMwOWY1N2Q4NTI4In0sICJQcmljZSI6ICJSTSAyLDUzOS4wMSIsICJTaGFwZSI6ICJQZWFyIiwgIkNhcmF0IjogIjAuMzEwIiwgIkNsYXJpdHkiOiAiSUYiLCAiQ29sb3IiOiAiRiIsICJDdXQiOiAiRVgiLCAiUG9saXNoIjogIlZHIiwgIlN5bW1ldHJ5IjogIlZHIiwgIkZsdW9yZXNjZW5jZSI6ICJOb25lIiwgIkNlcnRpZmljYXRlIE5vIjogIioqKioqKjYyMzUiLCAiTGFiIjogIkdJQSIsICJWaWV3IjogIk1vcmUgSW5mb3JtYXRpb24ifQ==', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-----+-----+-------+-----+------+--------+------------+
|Price  |Shape|Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-----+-----+-------+-----+------+--------+------------+
|2

...

Data saved to file
Processed record number: 1306
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDUxMmQxZGMwOWY1N2Q4NjQ1In0sICJQcmljZSI6ICJSTSAyLDE3NS4zNSIsICJTaGFwZSI6ICJBc3NjaGVyIiwgIkNhcmF0IjogIjAuMzAwIiwgIkNsYXJpdHkiOiAiVlZTMiIsICJDb2xvciI6ICJGIiwgIkN1dCI6ICJFWCIsICJQb2xpc2giOiAiRVgiLCAiU3ltbWV0cnkiOiAiVkciLCAiRmx1b3Jlc2NlbmNlIjogIk5vbmUiLCAiQ2VydGlmaWNhdGUgTm8iOiAiKioqKioqNTUwOCIsICJMYWIiOiAiR0lBIiwgIlZpZXciOiAiTW9yZSBJbmZvcm1hdGlvbiJ9', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-------+-----+-------+-----+------+--------+------------+
|Price  |Shape  |Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-------+-----+-------+-----+------+--------+----

Data saved to file
Processed record number: 1311
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDUxMmQxZGMwOWY1N2Q4NjRhIn0sICJQcmljZSI6ICJSTSAyLDE5NS4xOCIsICJTaGFwZSI6ICJBc3NjaGVyIiwgIkNhcmF0IjogIjAuMzAwIiwgIkNsYXJpdHkiOiAiVlMxIiwgIkNvbG9yIjogIkYiLCAiQ3V0IjogIlZHIiwgIlBvbGlzaCI6ICJFWCIsICJTeW1tZXRyeSI6ICJFWCIsICJGbHVvcmVzY2VuY2UiOiAiTm9uZSIsICJDZXJ0aWZpY2F0ZSBObyI6ICIqKioqKio2NDEyIiwgIkxhYiI6ICJHSUEiLCAiVmlldyI6ICJNb3JlIEluZm9ybWF0aW9uIn0=', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-------+-----+-------+-----+------+--------+------------+
|Price  |Shape  |Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-------+-----+-------+-----+------+--------+----

Data saved to file
Processed record number: 1318
Preprocessing data: {'$binary': {'base64': 'eyJfaWQiOiB7IiRvaWQiOiAiNjZkZDhiNDUxMmQxZGMwOWY1N2Q4NjUxIn0sICJQcmljZSI6ICJSTSAyLDIyMS42MyIsICJTaGFwZSI6ICJBc3NjaGVyIiwgIkNhcmF0IjogIjAuMzEwIiwgIkNsYXJpdHkiOiAiVlMxIiwgIkNvbG9yIjogIkgiLCAiQ3V0IjogIkVYIiwgIlBvbGlzaCI6ICJWRyIsICJTeW1tZXRyeSI6ICJWRyIsICJGbHVvcmVzY2VuY2UiOiAiTm9uZSIsICJDZXJ0aWZpY2F0ZSBObyI6ICIqKioqKioyODUyIiwgIkxhYiI6ICJHSUEiLCAiVmlldyI6ICJNb3JlIEluZm9ybWF0aW9uIn0=', 'subType': '00'}}
root
 |-- Price: double (nullable = true)
 |-- Shape: string (nullable = true)
 |-- Carat: double (nullable = true)
 |-- Clarity: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Polish: string (nullable = true)
 |-- Symmetry: string (nullable = true)
 |-- Fluorescence: string (nullable = true)

+-------+-------+-----+-------+-----+------+--------+------------+
|Price  |Shape  |Carat|Clarity|Color|Polish|Symmetry|Fluorescence|
+-------+-------+-----+-------+-----+------+--------+----

ERROR:kafka.coordinator.consumer:Offset commit failed: This is likely to cause duplicate message delivery
Traceback (most recent call last):
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/kafka/coordinator/consumer.py", line 537, in _maybe_auto_commit_offsets_sync
    self.commit_offsets_sync(self._subscription.all_consumed_offsets())
  File "/home/student/de-prj/de-venv/lib/python3.10/site-packages/kafka/coordinator/consumer.py", line 530, in commit_offsets_sync
    raise future.exception # pylint: disable-msg=raising-bad-type
kafka.errors.CommitFailedError: CommitFailedError: Commit cannot be completed since the group has already
            rebalanced and assigned the partitions to another member.
            This means that the time between subsequent calls to poll()
            was longer than the configured max_poll_interval_ms, which
            typically implies that the poll loop is spending too much
            time message processing. You can address this 

Data saved to file
Processed record number: 1320
Reached 1320 records. Stopping consumption.
Consumer and Spark session closed.


In [12]:
# Store Csv 